In [1]:
from dataclasses import dataclass
import pandas as pd
import os
import SimpleITK as sitk

import keras as tfk
from keras import layers as tfkl

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.data import Dataset as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

sns.set_theme()

In [2]:
SEED = 42
BATCH_SIZE = 64
IMG_HEIGHT = 52
IMG_WIDTH = 52
SHUFFLE_BUFFER_SIZE = 100
METRICS = ["accuracy", "recall", "f1_score"]

data_dir = '/kaggle/input/lung-ds-norm/Nodule'
more_data_dir = '/kaggle/input/smote-ds-nodules/smote-ds-nodule'

tfk.utils.set_random_seed(SEED)

In [3]:
import os
import shutil

# Define writable working directory
working_dir = "/kaggle/working/Nodule"

# Copy the dataset to the working directory (if not already copied)
if not os.path.exists(working_dir):
    shutil.copytree(data_dir, working_dir)
    print("Dataset copied to writable directory.")

# Loop over both "train" and "val" folders
for folder in ["train", "val"]:
    folder_path = os.path.join(working_dir, folder)

    # Move files from 1 and 2 → 0
    for subfolder in ["1", "2"]:
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.exists(subfolder_path):
            for file in os.listdir(subfolder_path):
                shutil.move(os.path.join(subfolder_path, file), os.path.join(folder_path, "0"))

    # Move files from 3 and 4 → 1
    for subfolder in ["3", "4"]:
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.exists(subfolder_path):
            for file in os.listdir(subfolder_path):
                shutil.move(os.path.join(subfolder_path, file), os.path.join(folder_path, "1"))

    # Delete folders 2, 3, and 4
    for subfolder in ["2", "3", "4"]:
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.exists(subfolder_path):
            shutil.rmtree(subfolder_path)

print("Files moved and folders deleted successfully!")

Dataset copied to writable directory.
Files moved and folders deleted successfully!


In [4]:
# Add SMOTE data
smote_working_dir = "/kaggle/working/SMOTE"

# Copy the dataset to the working directory (if not already copied)
if not os.path.exists(smote_working_dir):
    shutil.copytree(more_data_dir, smote_working_dir)
    print("Dataset copied to writable directory.")

for folder in ["3", "4"]:
    folder_path = os.path.join(smote_working_dir, folder)
    if os.path.exists(folder_path):  # Check if the folder exists
        for file_name in os.listdir(folder_path):  # Iterate over files
            file_path = os.path.join(folder_path, file_name)
            shutil.move(file_path, working_dir+'/train/1')  # Move file

Dataset copied to writable directory.


In [5]:
# Compute the number of samples per class
for folder in ["train", "val"]:
    folder_path = os.path.join(working_dir, folder)
    print(f"\nFolder: {folder}")

    # Count files in each remaining subfolder (0 and 1)
    for subfolder in ["0", "1"]:
        subfolder_path = os.path.join(folder_path, subfolder)
        if os.path.exists(subfolder_path):
            num_files = len(os.listdir(subfolder_path))
            print(f"  {subfolder}: {num_files} images")
        else:
            print(f"  {subfolder}: Does not exist")


Folder: train
  0: 2661 images
  1: 2110 images

Folder: val
  0: 615 images
  1: 410 images


In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=30,            
    # width_shift_range=0.2,        
    # height_shift_range=0.2,       
    shear_range=0.2,              
    zoom_range=0.2,               
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant',
    cval=0    
)

val_datagen = ImageDataGenerator(

)

train_generator = train_datagen.flow_from_directory(
    working_dir + "/train",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
    #subset='training'               
)

val_generator = val_datagen.flow_from_directory(
    working_dir + "/val",
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
    #subset='val'
)

Found 4771 images belonging to 2 classes.
Found 1025 images belonging to 2 classes.


In [7]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Get unique class labels
classes = np.array([0, 1])

# Compute class weights automatically
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=train_generator.classes)

# Convert to dictionary
class_weights = dict(zip(classes, class_weights))

print(class_weights)

{0: 0.8964674934235249, 1: 1.130568720379147}


In [ ]:
# Get one batch of augmented images
augmented_images, _ = next(train_generator)  # Ignore labels (_)

# Plot a 5x5 grid of augmented images
fig, axes = plt.subplots(5, 5, figsize=(12, 12))
axes = axes.flatten()

for img, ax in zip(augmented_images[:25], axes):  # Take the first 25 images
    ax.imshow(img[:, :, 0], cmap='gray')
    ax.axis('off')

plt.tight_layout()
plt.show()

In [15]:
@dataclass
class Hyperparameters:
    # Model
    activation = "relu"
    # Training
    # noise_std: float = 0.05 # then try 0.1
    optimiser = tfk.optimizers.Adam
    learning_rate = 1e-4
    weight_decay = 1e-4
    regularization1 = tfk.regularizers.L1L2(l1=1e-5, l2=0)
    regularization2 = tfk.regularizers.L1L2(l1=0, l2=0)
    loss = tfk.losses.BinaryCrossentropy(from_logits=False)
    epochs = 50
    ## Early stopping parameters
    es_patience = 10
    es_min_delta = 1e-2
    ## Learning rate schedule
    lr_patience = 5
    lr_decay_factor = 0.3
    lr_min_delta = 5e-2 
    min_lr = 1e-6

hp = Hyperparameters()

In [16]:
def build_model(hp: Hyperparameters,
                feature_extractor: tfk.applications):
    
    inputs = tfkl.Input((IMG_HEIGHT, IMG_WIDTH, 3))

    # x = tfkl.Rescaling(1./255)(inputs)
    # gn = tfkl.GaussianNoise(hp.noise_std)(x)
    # x = tfkl.Rescaling(255)(gn)

    backbone_output = feature_extractor(inputs)

    conv_layer = tfkl.Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same')(backbone_output) 
    max_pool_layer = tfkl.MaxPooling2D(pool_size=(2, 2))(conv_layer)
    flatten_layer = tfkl.Flatten()(max_pool_layer) 

    x = tfkl.Dropout(0.6)(flatten_layer)

    x = tfkl.Dense(1024, activation='relu')(x)
    x = tfkl.Dense(512, activation='relu')(x)
    x = tfkl.Dense(256, activation='relu')(x)
    #x = tfkl.Dense(128, activation='relu')(x)

    x = tfkl.Dropout(0.4)(x)

    output = tfkl.Dense(1, activation="sigmoid")(x)
    
    model = tfk.Model(inputs, output)
    return model

In [20]:
def fit(model: tfk.Model,
        train_generator: ImageDataGenerator,
        val_generator: ImageDataGenerator,
        hp: Hyperparameters):
    model.compile(loss=hp.loss,
                  optimizer=hp.optimiser(learning_rate=hp.learning_rate, weight_decay=hp.weight_decay), 
                  metrics=['accuracy'])
    
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=hp.epochs,
        class_weight=class_weights,
        #validation_data=val_dataset,
        callbacks=[
            tfk.callbacks.EarlyStopping(monitor='val_loss', 
                                        #mode='min',
                                        patience=hp.es_patience, 
                                        restore_best_weights=True),
            tfk.callbacks.ReduceLROnPlateau(
                factor=hp.lr_decay_factor,
                patience=hp.lr_patience,
                min_delta=hp.lr_min_delta,
                min_lr=hp.min_lr,
                verbose=1,
            )
    ]
    
    ).history

    return model, history

In [21]:
backbone = tfk.applications.EfficientNetB0(
    include_top=False,
    weights='/kaggle/input/efficientnetb0/keras/default/1/efficientnetb0_notop.h5',
    input_tensor=None,
    input_shape=(IMG_HEIGHT,IMG_WIDTH,3),
    pooling=None #'avg'
)

backbone.trainable = False

model = build_model(hp, backbone)

model, history = fit(model, train_generator, val_generator, hp)

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


75/75 ━━━━━━━━━━━━━━━━━━━━ 189s 2s/step - accuracy: 0.6430 - loss: 0.6229 - val_accuracy: 0.7268 - val_loss: 0.5073 - learning_rate: 1.0000e-04
Epoch 2/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.7548 - loss: 0.4931 - val_accuracy: 0.7395 - val_loss: 0.5090 - learning_rate: 1.0000e-04
Epoch 3/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 173s 2s/step - accuracy: 0.7868 - loss: 0.4727 - val_accuracy: 0.7502 - val_loss: 0.4948 - learning_rate: 1.0000e-04
Epoch 4/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.7812 - loss: 0.4672 - val_accuracy: 0.7610 - val_loss: 0.4892 - learning_rate: 1.0000e-04
Epoch 5/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 176s 2s/step - accuracy: 0.7813 - loss: 0.4517 - val_accuracy: 0.7327 - val_loss: 0.5011 - learning_rate: 1.0000e-04
Epoch 6/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7815 - loss: 0.4404
Epoch 6: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
75/75 ━━━━━━━━━━━━━━━━━━━━ 177s 2s/step - accuracy: 0.7817 - loss: 0.4404 - 

In [22]:
# Plot loss curves
plt.style.use('default')
plt.plot(history['loss'], label='Train loss', alpha=0.3, linestyle='--')
plt.plot(history['val_loss'], label='Val loss', alpha=0.8)
plt.title('Binary Crossentropy Loss')
plt.legend(loc='upper left')
plt.grid(alpha=0.3)

In [23]:
# Plot accuracy curves
plt.style.use('default')
plt.plot(history['accuracy'], label='Train accuracy', alpha=0.3, linestyle='--')
plt.plot(history['val_accuracy'], label='Val accuracy', alpha=0.8)
plt.title('Accuracy')
plt.legend(loc='upper left')
plt.grid(alpha=0.3)

17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 335ms/step
Accuracy:  0.76
Precision:  0.65
Recall:  0.85
F1 score:  0.74


In [24]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Collect true labels
y_true = []
for _, labels in val_generator:
    y_true.extend(labels)
    if len(y_true) >= val_generator.samples:  # Stop when all samples are processed
        break

y_true = np.array(y_true)

# If class_mode is 'binary', labels might have an extra dimension (shape: (batch_size, 1)), so we flatten
if val_generator.class_mode == 'binary':
    y_true = y_true.flatten()

# Predict on validation data
y_pred = model.predict(val_generator)

# Convert probabilities to class labels (0 or 1)
y_pred_classes = (y_pred > 0.5).astype(int).flatten()

# Print accuracy
print("Accuracy: ", round(accuracy_score(y_true, y_pred_classes), 2))
print("Precision: ", round(precision_score(y_true, y_pred_classes), 2))
print("Recall: ", round(recall_score(y_true, y_pred_classes), 2))
print("F1 score: ", round(f1_score(y_true, y_pred_classes), 2))

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[0, 1], yticklabels=[0, 1])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
